# Задача 7. Нейронные сети

## Задача

- [ ] Найти данные в виде изображений для задачи классификации.
- [ ] Реализовать классы, необходимые для построения сети со следующими слоями
	- FullyConnectedLayer
	- ReluLayer
	- FullyConnectedLayer
- [ ] Использовать CrossEntropyLoss и L2-регуляризацию.
- [ ] Обучить модель на тренировочных данных, подбирать параметры (особенно learning rate) на валидационной и оценить качество на тестовой. Анализировать графики train/val loss, проверять на каждом шаге корректность вычисления градиентов с помощью разностной оценки.
- [ ] * (**+2 балла**) Добавить Batch normalization.
- [ ] * (**+2 балла**) В качестве оптимизатор использовать один из: Momentum, RMSprop.
- [ ] * (**+1 балл**) Также реализовать оптимизатор Adam.

## Датасет

[Butterfly Image Classification](https://www.kaggle.com/datasets/phucthaiv02/butterfly-image-classification) — содержит более 10 000 изображений 75 различных видов бабочек.

Каждое изображение имеет размер 224x224 пикселя. Класс каждого изображения хранится в файле Training_set.csv.